In [40]:
import sys
print(sys.version)

3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [41]:
import tobii_research as tr

In [42]:
import time
import csv
import os
import keyboard
from collections import defaultdict

Inicializo diccionario para contar los eventos (pulsar la tecla e)

In [43]:
event_count = defaultdict(int)
key_pressed = False

In [44]:
 global_user_position_guide = None

In [45]:
global_csv_filename = "gaze_data_1.csv"

In [46]:
found_eyetrackers = tr.find_all_eyetrackers()

In [36]:
my_eyetracker = found_eyetrackers[0]

IndexError: tuple index out of range

In [37]:
print("Address: " + my_eyetracker.address)
print("Model: " + my_eyetracker.model)
print("Name: " + my_eyetracker.device_name)
print("Serial number: " + my_eyetracker.serial_number)

NameError: name 'my_eyetracker' is not defined

Función que saca las **posiciones X e Y de cada ojo** y lo escribe en un csv:

Iba a plantearla sobreescribiendo el archivo de datos cada vez que se analice, pero la he dejado así por si queremos tener diferentes archivos con datos particulares, simplemente cambiar el nombre del csv a crear y así podremos crear distintos csvs con distintos datos con la misma función.

**IMPORTANTE: BORRAR EL CSV ANTES DE HACER QUE ESTA FUNCIÓN SE EJECUTE**

In [47]:
def gaze_data_callback(gaze_data):
    
    global event_count, key_pressed, global_csv_filename
    
    if keyboard.is_pressed('e') and not key_pressed:
        event_count['e']+=1
        key_pressed = True
    elif not keyboard.is_pressed('e'):
        key_pressed = False

    csv_filename = global_csv_filename

    fieldnames = ['Left Eye X', 'Left Eye Y', 'Right Eye X', 'Right Eye Y', 'Pupil left eye', 'Pupil right eye', 'Timestamp device', 'Timestamp system', 'Event']
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        if file.tell() == 0:
            writer.writeheader()

        left_eye_data = gaze_data['left_gaze_point_on_display_area']
        right_eye_data = gaze_data['right_gaze_point_on_display_area']
        left_eye_pupil = gaze_data['left_pupil_diameter']
        right_eye_pupil = gaze_data['right_pupil_diameter']
        timestamp_device = gaze_data['device_time_stamp']
        timestamp_system = gaze_data['system_time_stamp']

        left_eye_data = tuple(map(float, left_eye_data))
        right_eye_data = tuple(map(float, right_eye_data))
        
        event_number = event_count['e']

        row_dict = {
            'Left Eye X': left_eye_data[0],
            'Left Eye Y': left_eye_data[1],
            'Right Eye X': right_eye_data[0],
            'Right Eye Y': right_eye_data[1],
            'Pupil left eye': left_eye_pupil,
            'Pupil right eye': right_eye_pupil,
            'Timestamp device': timestamp_device,
            'Timestamp system': timestamp_system,
            'Event': f'Evento {event_number}'
        }

        writer.writerow(row_dict)

In [48]:
my_eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)


NameError: name 'my_eyetracker' is not defined

In [21]:
#time.sleep(5)


In [22]:
my_eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)

NameError: name 'my_eyetracker' is not defined